# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Модель классификации комментариев на позитивные и негативные позволит найти негативные для отправки их на модерацию. Имеется набор данных с разметкой о токсичности правок.

Построим разные модели и выясним какая дает значеник метрики качества F1 не меньше 0.75.

План по проекту:
1. Загрузка и подготовка данных.
2. Обучение различных моделей.
3. Выводы.

Данные находятся в файле toxic_comments.csv. Столбец text содержит текст комментария, а toxic — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Общая-информация" data-toc-modified-id="Общая-информация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Общая информация</a></span></li><li><span><a href="#Вывод-по-полученным-данным" data-toc-modified-id="Вывод-по-полученным-данным-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вывод по полученным данным</a></span></li><li><span><a href="#Подготовка-текста" data-toc-modified-id="Подготовка-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка текста</a></span></li><li><span><a href="#Разбиение-на-выборки" data-toc-modified-id="Разбиение-на-выборки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разбиение на выборки</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Финальное-тестирование.-Проверим-качество-моделей-на-тестовой-выборке" data-toc-modified-id="Финальное-тестирование.-Проверим-качество-моделей-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Финальное тестирование. Проверим качество моделей на тестовой выборке</a></span></li></ul></div>

## Подготовка данных

In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import math 
pd.options.mode.chained_assignment = None

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

from IPython.display import display
from tqdm import notebook 
from tqdm.notebook import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import time
from datetime import timedelta
import matplotlib.pyplot as plt

#библиотеки для преобразования текста
import re
import string
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

#библиотеки для модели
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, cross_val_score, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier
from sklearn.utils import shuffle

#BERT
import torch
import transformers 

import warnings
warnings.filterwarnings('ignore')

/tmp/ipykernel_48/2978543179.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Включаем полное отображение содержимого столбцов

In [8]:
pd.set_option('display.max_colwidth', -1)

Получаем таблицу с данными

https://drive.google.com/file/d/1W8MwyoSKBY8lkxP9JDWwJj2Cid-g5Phz/view?usp=sharing

https://drive.google.com/file/d/1iercy90CiB05Rnjc4Lh6-bQp0vclnMJ3/view?usp=sharing

In [11]:
!gdown --id 1iercy90CiB05Rnjc4Lh6-bQp0vclnMJ3

/bin/bash: gdown: command not found


In [12]:
df = pd.read_csv('D:\Мусаев\1\IT\2_yandex.practikum\уроки\проект\13_мл_для_текстов\toxic_comments.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Мусаев\x01\\IT\x02_yandex.practikum\\уроки\\проект\x0b_мл_для_текстов\toxic_comments.csv'

### Общая информация

In [ ]:
df.head()

In [ ]:
df.info()

Посмотрим на распределение целевого признака.

In [ ]:
df['toxic'].value_counts(normalize=True)

Видим, что классы несбалансированные.

In [ ]:
df.isna().sum()

Пропуски отсутствуют

In [ ]:
df.duplicated().sum()

Дубликаты отсутствуют

### Вывод по полученным данным

1. Столбец "text" - текст с комментариями. Тип данных object

2. Столбец "toxic" - целевой показатель, оценка токсичности, 0 - позитив, 1 - негатив

3. Таблица имеет 159571 объектов

4. Классы несбалансированные.

5. Отсутствуют пропуски и дубликаты







### Подготовка текста

Приступаем к подготовке текстовых данных столбца 'text' для обучения моделей - переведем их в векторы при помощи модели BERT

До перевода текста в векторы токенезируем их. Инициализируем токенизатор как объект класса BertTokenizer(). Передадим ему аргумент vocab_file — это файл со словарём, на котором обучалась модель. Он может быть, например, в текстовом формате (txt).

Инициализируем токенизатор как объект класса BertTokenizer(). Передадим ему аргумент vocab_file — это файл со словарём. Ниже ссылка и код загрузки 

In [ ]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'distilbert-base-uncased')

In [ ]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

BERT не может принимать на вход последовательности длиннее, чем 512 токенов. Присваиваем max_length = 512

Преобразуем текст в номера токенов из словаря методом encode()


<div class="alert alert-block alert-info">

Перед лемматизацией добавлена очистка текста от лишних символов
</div>

In [ ]:
def clear_text(text):
    # < напишите код здесь >
    new_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    cleared_text = " ".join(new_text.split())
    return cleared_text

In [ ]:
cleared_text = df['text'].progress_apply(lambda x: clear_text(x))

In [ ]:
%%time
tokenized = cleared_text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [ ]:
max_length = 512

Методом padding дополним нулями строки до самой длинной, получим строки одинаковой длины. 

In [ ]:
padded = np.array([i + [0]*(max_length - len(i)) for i in tokenized.values])

Для экономии времени и ресурсов сохраним padded на диске



In [ ]:
%%time
np.savetxt("padded_csv", padded, delimiter=",")

https://drive.google.com/file/d/1BzYaVW3nk_HHnPGHuevUp-qMU4wh-W7-/view?usp=sharing

In [ ]:
!gdown --id 1BzYaVW3nk_HHnPGHuevUp-qMU4wh-W7-

In [ ]:
%%time
padded = np.array(pd.read_csv('/content/padded_csv'))

Теперь поясним модели, что нули не несут значимой информации. Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape) 

Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим

In [ ]:
batch_size = 100

Сделаем цикл по батчам. Отображать прогресс будет функция notebook(). Для сокращения времени создания эмбедингов возьмем выборку 1000 строк.

In [ ]:
padded = padded[:1000]
padded.shape

<div class="alert alert-block alert-success">
✔️ Вынужденное снижение размера выборки из-за ограниченных вычислительных возможностей может негативно сказаться на качестве модели.
</div>

In [ ]:
%%time

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy()) 

In [ ]:
features = np.concatenate(embeddings)
features = pd.DataFrame(features)
features.shape

### Разбиение на выборки

Для сокращения времени возьмем выборку 1000 строк.:

In [ ]:
df = df.head(1000)

Разделяем исходные данные на обучающую 80% и валидационную выборку 20%

In [ ]:
labels =df['toxic']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.2, random_state=42)

Проверяем размер данных

In [ ]:
len(df) - len(x_train) - len(x_test)

In [ ]:
len(df) - len(y_train) - len(y_test)

Целевой признак обучающий и тестовый

В результате разделения получили данные:

- обучающие признаки

- обучающие целевые признаки

- тестовые признаки

- тестовые целевые признаки

## Обучение моделей

Обучим четыре модели: 
- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression
- CatBoostClassifier

### Решающее дерево

Построим модель решающее дерево и с помощью функции GridSearchCV подберем параметры.

Создадим через функцию KFold условия для выборок в кросс-валидации.

In [ ]:
shuffle = KFold(n_splits=5, random_state=42, shuffle=True)

Построим модель, параметр class_weight укажем 'balanced'.

In [ ]:
%%time
DecisionTreeClassifier = DecisionTreeClassifier(random_state=42, 
                                                class_weight='balanced')

<div class="alert alert-block alert-success">
✔️ class_weight='balanced' позволит учесть дисбаланс
</div>

Создадим список перебираемых параметров и передадим их в GridSearchCV.

In [ ]:
param = {'max_depth': np.arange(2, 10, 1),
          'min_samples_split': np.arange(1, 4, 1),
          'min_samples_leaf': np.arange(1, 6, 1)}

#подбирает гиперпараметры
model_DecisionTreeClassifier = GridSearchCV(DecisionTreeClassifier, 
                                            param, 
                                            scoring='f1', 
                                            cv=shuffle)

model_DecisionTreeClassifier.fit(x_train, y_train)

print('F1-мера: {:.3f}'.format(model_DecisionTreeClassifier.best_score_))
print('Лучшие параметры', model_DecisionTreeClassifier.best_params_)

Обучена модель DecisionTreeClassifier, настроены гиперпараметы, получены предсказания и оценка качества F1 значительно ниже целевой

<div class="alert alert-block alert-success">
✔️ Глубина дерева найдена на границе сетки. Видимо, дерево совсем быстро переобучается на эмбеддингах, если лучшее оказалось с минимальной глубиной.
</div>

### Случайный лес

In [ ]:
%%time
RandomForestClassifier = RandomForestClassifier(random_state=42, 
                                                class_weight='balanced')

In [ ]:
param = {
          'max_depth': np.arange(5, 9, 1),
          'min_samples_split': np.arange(0, 4, 1),
          'min_samples_leaf': np.arange(3, 7, 1),
          'n_estimators': np.arange(1, 5, 1)
          }

#подбирает гиперпараметры
model_RandomForestClassifier = GridSearchCV(RandomForestClassifier, 
                                            param, 
                                            scoring='f1', 
                                            cv=shuffle)

model_RandomForestClassifier.fit(x_train, y_train)

print('F1-мера: {:.3f}'.format(model_RandomForestClassifier.best_score_))
print('Лучшие параметры', model_RandomForestClassifier.best_params_)

Обучена модель RandomForestClassifier, настроены гиперпараметы, получены предсказания и оценка качества F1 значительно ниже целевой

<div class="alert alert-block alert-success">
✔️ Даже случайный лес переобучается. Обычно в лесу деревья повыше дают лучший результат.
</div>

### Логистическая регрессия

In [ ]:
%%time
model_LogisticRegression = LogisticRegression(random_state=42, 
                                              solver='liblinear',
                                              penalty='l2',
                                              C=5,
                                              class_weight='balanced')

model_LogisticRegression.fit(x_train, y_train)
predicted_LogisticRegression = model_LogisticRegression.predict(x_test)

In [ ]:
f1_val_LogisticRegression = f1_score(y_test, predicted_LogisticRegression)
print("значение метрики качества F1 = {:.3f}".format(f1_val_LogisticRegression))

Обучена модель LogisticRegression, настроены гиперпараметы, получены предсказания и оценка качества F1 значительно ниже целевой

### CatBoostClassifier

Для модели CatBoostClassifier требуется валидационная выборка для защиты от переобучения. От выборки train возьмем 25% для получения пропорций 60-20-20

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.2, random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.25, random_state=42)

Проверяем размер данных

In [ ]:
len(df) - len(x_train) - len(x_test) - len(x_val)

In [ ]:
len(df) - len(y_train) - len(y_test) - len(y_val)

В результате разделения получили данные:

- обучающие признаки

- обучающие целевые признаки

- валидационные признаки

- валидационные целевые признаки

- тестовые признаки

- тестовые целевые признаки

Модель CatBoostClassifier будем строить с подбором гиперпараметры с помощью функции GridSearchCV.

Зададим веса для несбалансированных классов.

In [ ]:
scale_pos_weight = round((len(y_train[y_train == 0]) / 
                          len(y_train[y_train == 1])), 3)
scale_pos_weight

In [ ]:
%%time
modelBoostClassifier = CatBoostClassifier(random_state = 42, verbose=100, eval_metric='F1',
                              scale_pos_weight=scale_pos_weight)

modelBoostClassifier.fit(x_train, y_train,eval_set=(x_val, y_val))

<div class="alert alert-block alert-success">
✔️ По логам видно, как модель быстро переобучилась.
</div>

In [ ]:
predictedBoostClassifier = modelBoostClassifier.predict(x_val)

In [ ]:
f1_val_predictedBoostClassifier = f1_score(y_val, predictedBoostClassifier)
print("значение метрики качества F1 = {:.3f}".format(f1_val_predictedBoostClassifier))

Обучена модель CatBoostClassifier, настроены гиперпараметы, получены предсказания и оценка качества F1 также значительно ниже целевой

## Финальное тестирование. Проверим качество моделей на тестовой выборке

In [ ]:
predicted_DecisionTreeClassifier = model_DecisionTreeClassifier.predict(x_test)
f1_test_predicted_DecisionTreeClassifier = f1_score(y_test, predicted_DecisionTreeClassifier)

In [ ]:
predicted_RandomForestClassifier = model_RandomForestClassifier.predict(x_test)
f1_test_predicted_RandomForestClassifier = f1_score(y_test, predicted_RandomForestClassifier)

In [ ]:
predicted_LogisticRegression = model_LogisticRegression.predict(x_test)
f1_test_predicted_LogisticRegression = f1_score(y_test, predicted_LogisticRegression)

In [ ]:
predictedBoostClassifier = modelBoostClassifier.predict(x_test)
f1_test_predictedBoostClassifier = f1_score(y_test, predictedBoostClassifier)

In [ ]:
df_tab = pd.DataFrame({
        'модель':     ['DecisionTreeClassifier', 'RandomForestClassifier', 
                      'LogisticRegression', 'CatBoostClassifier'], 
        'f1 - val': [model_DecisionTreeClassifier.best_score_, 
                      model_RandomForestClassifier.best_score_, 
                      f1_val_LogisticRegression,
                      f1_val_predictedBoostClassifier],
        'f1 - test': [f1_test_predicted_DecisionTreeClassifier, 
                      f1_test_predicted_RandomForestClassifier, 
                      f1_test_predicted_LogisticRegression,
                      f1_test_predictedBoostClassifier]})
df_tab

**Вывод:**

 
1. Загружены данные с характеристикам:

  - Столбец 1 - текст с комментариями. Тип данных object

  - Столбец 2 - целевой показатель, оценка токсичности, 0 - позитив, 1 - негатив

  - Таблица имеет 159571 объектов

  - Классы несбалансированные

  - Отсутствуют пропуски и дубликаты
  

2. Для сокращения времени создания эмбедингов взята выборка 1000 строк.Для векторного представления текста и обучения моделей бинарной классификации
выполнено:

  - токенезация текста

  - преобразование текстов в эмбеддинги

  - все эмбеддинги собираем в матрицу признаков
  

3. Обучены четыре модели: 

  - DecisionTreeClassifier

  - RandomForestClassifier

  - LogisticRegression

  - CatBoostClassifier
  

4. Все модели показали недостаточный уровень качества - менее 0.4, при целевом - 0.75. Вероятно из-за выборки всего 1000 объектов от 159000 - 0.6%, т.к. ресурсы и не позволили провести работу с полной таблицей.


5. По скорости обучения менее 1 сек и показателю качества f1 = 0.431 лучшей оказалась LogisticRegression. Вероятно, на полных данных модель позволит получить уровень качества выше 0.75

<div class="alert alert-block alert-success">
✔️ Да, маленькая выборка скорее всего привела к быстрому переобучению модели - она быстро подстроилась под обучающую выборку, но на тестовой показывает очень низкий результат. Т.е. эмбеддинги хороши на большой выборке и их лучше дообучать на конкретной предметной области.<br>
✔️ В этой задаче по метрике вполне бы зашла логистическая регрессия с лемматизацией, TF-IDF, n-граммами до 2-3 слов и подбором гиперпараметров, особенно типа (penalty) и степени регуляризации (С).<br><br>
</div>